# How to use
分類問題でどのように使用するかを説明します。   

## install
以下のコードをnb_sissoをinstallしたい環境で実行

In [2]:
#!pip install git+https://github.com/souno1218/nb_sisso.git

## データ作成

まず(randomな)データを作成します。   
この時、初期特徴量は`a~f`で6種類(`n_features=6`)、サンプル数は220(`n_samples=220`)、目標変数は`target`としています。   
また、目標変数は2値で、`[True,False]`とします。   

In [3]:
import numpy as np
import pandas as pd
rng = np.random.default_rng()
x=rng.random((6,220))
y=rng.choice((True,False),220)
columns=[f"{chr(i + 97)}" for i in range(x.shape[0])]
index=[f"sample_{i}" for i in range(x.shape[1])]
df=pd.DataFrame(x.T,columns=columns,index=index)
df["target"]=y
df.head()

,a,b,c,d,e,f,target
sample_0,0.622239,0.344505,0.727774,0.361563,0.030092,0.733566,True
sample_1,0.877103,0.636877,0.971538,0.809459,0.723680,0.602633,True
sample_2,0.164651,0.023225,0.454471,0.109348,0.875270,0.715368,True
sample_3,0.130880,0.368716,0.781383,0.722587,0.717695,0.318759,True
sample_4,0.294608,0.058915,0.028766,0.997474,0.398430,0.777796,False


単位系をそれぞれ`{a: "m/s", b: "m^2", c: "m/s^2", d: "a.u.", e: "m", f: "a.u."}`とします。   
この時単位系は2種類なので、以下のようなshapeが`(n_features,2)`なndarrayを作成します。   
この時、dtypeはint64である必要があります。   

In [4]:
units=np.zeros((x.shape[0],2),dtype="int64")
units[0,0] = 1  # a:    "m"
units[0,1] = -1 # a:   "/s"
units[1,0] = 2  # b:  "m^2"
units[1,1] = 0  # b: "no s"
units[2,0] = 1  # c:    "m"
units[2,1] = -2 # c: "/s^2"
units[3,0] = 0  # d: "no m"
units[3,1] = 0  # d: "no s"
units[4,0] = 1  # e:    "m"
units[4,1] = 0  # e: "no s"
units[5,0] = 0  # f: "no m"
units[5,1] = 0  # f: "no s"
units_df=pd.DataFrame(units.T,columns=columns,index=["m","s"])
units_df

,a,b,c,d,e,f
m,1,2,1,0,1,0
s,-1,0,-2,0,0,0


各配列のshapeを見ておきます。

In [5]:
print(f"x.shape={x.shape}, y.shape={y.shape}, units.shape={units.shape}")

x.shape=(6, 220), y.shape=(220,), units.shape=(6, 2)


## SIS

使用する演算子を決めます。以下が使用可能です。   
`["+","-","*","/","*-1","**-1","**2","sqrt","| |","**3","cbrt","**6","exp","exp-","log","sin","cos"]`

In [6]:
operators_to_use=["+","-","*","/","**2","| |","**3"]

model_scoreを決めます。2値分類に関してはさまざま作ってありますが、今のところ多値分類,回帰はありません。   
上のような`x,y`のみを引数に取り、float64の値を2つ返すことができれば、それだけで良いので、pythonがわかればある程度簡単にオリジナルのものを作ることができます。   

In [7]:
from nb_sisso.model_score_1d import Hull_1d
model_score=Hull_1d

SISを走らせます。その際、初期特徴量として`np.ones(n_samples)`を含めるかを`is_use_1`で設定できます。   
`is_use_1 = True`でも、`x`に`np.ones(n_samples)`を加える必要はありません。   
今回は保存数5000(`how_many_to_save=5000`),最大演算子数4(`max_n_op=4`),`is_use_1=True`とします。   
初回実行時はコンパイルが入るため、動き出すまでが遅いです。   
いくつかのオプション引数があります。nb_SO.pyをご覧ください。   

In [8]:
from nb_sisso import SIS
score,eq=SIS(x,y,model_score=model_score,units=units,how_many_to_save=5000,is_use_1=True,max_n_op=4,operators_to_use=operators_to_use)

2024-11-06 17:12:50,787 SIS [INFO] : numba=0.60.0, numpy=2.0.2
2024-11-06 17:12:50,787 SIS [INFO] : OPT=_OptLevel(3), THREADING_LAYER=default
2024-11-06 17:12:50,787 SIS [INFO] : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2024-11-06 17:12:50,787 SIS [INFO] : SIS
2024-11-06 17:12:50,788 SIS [INFO] : num_threads=8, how_many_to_save=5000, 
2024-11-06 17:12:50,788 SIS [INFO] : how_many_to_save_per_1_core=5000, 
2024-11-06 17:12:50,788 SIS [INFO] : max_n_op=4, model_score=Hull_1d, 
2024-11-06 17:12:50,788 SIS [INFO] : x.shape=(6, 220), is_use_1=True
2024-11-06 17:12:50,788 SIS [INFO] : use_binary_op=[-1, -2, -3, -4], 
2024-11-06 17:12:50,789 SIS [INFO] : use_unary_op=[-7, -9, -10]
2024-11-06 17:12:50,789 SIS [INFO] : units=[ 1 -1] , [2 0] , [ 1 -2] , [0 0] , [1 0] , [0 0]
2024-11-06 17:12:50,789 SIS [INFO] : compiling
2024-11-06 17:13:16,805 SIS [INFO] : END, compile
2024-11-06 17:13:16,806 SIS [INFO] :   n_op=1
2024-11-06 17:13:17,006 SIS [INFO] :     binary_op n_op1:n_op2 = 0:0,  lo

返り値は2つで、`score,eq`です。`score`のshapeは`(how_many_to_save,2)`で、`eq`と対応しています。   
scoreは2つあるためdim2は2になっています。   
`eq`のshapeは`(how_many_to_save,2*max_n_op+1)`で、数式を配列にしたもので、このまま読むことはできません。   
`decryption`関数に入れることで、読めるようになります。

In [9]:
from nb_sisso.utils import decryption
decryption(eq[0])

'(e/((d/(f**3))-f))'

In [10]:
df_SIS_ans=pd.DataFrame()
str_eq=[decryption(eq[i]) for i in range(eq.shape[0])]
df_SIS_ans["score1"]=score[:,0]
df_SIS_ans["score2"]=score[:,1]
df_SIS_ans["str_eq"]=str_eq
df_SIS_ans.head()

,score1,score2,str_eq
0,0.068182,-1.0,(e/((d/(f**3))-f))
1,0.063636,-1.0,((b/a)*((c**3)**2))
2,0.063636,-1.0,((b/a)*((c*c)**3))
3,0.063636,-1.0,((a/b)/((c**3)**2))
4,0.063636,-1.0,((a/b)/((c*c)**3))


## SO

SISの結果`how_many_to_save`個から2つ選ぶ組み合わせを全て行い、最もスコアが高いものを選ぶ。   


SISの結果からSO向けの特徴量を作成します。SISで使用した`x`と、SISの返り値の`eq`を`eq_list_to_num`に入れると、`eq`に従って計算された特徴量が返ってきます。   
SISでは特徴量はlistである必要があるため、listに変更します。   

In [11]:
from nb_sisso.utils import eq_list_to_num
X=eq_list_to_num(x,eq)
list_x=[X]

model_scoreを決めます。2値分類に関してはさまざま作ってありますが、今のところ多値分類,回帰はありません。   

In [13]:
from nb_sisso.model_score_2d import Hull_2d
model_score=Hull_2d

下のような`x,y`のみを引数に取り、float64の値を2つ返すことができれば、それだけで良いので、pythonがわかればある程度簡単にオリジナルのものを作ることができます。   
ここで、返り値2つはそれぞれscore1,score2とよび、score1で(大きい順に)ソートし、score1が同じもの同士ではscore2が大きいものを優位とします。   

In [18]:
one_X=X[[0,1]].T
print(one_X.shape)
model_score(one_X,y)

(220, 2)


(0.1681818181818182, -0.22820642720858972)

SOを走らせます。`which_arr_to_choose_from`の詳しい役割はnb_SO.pyを見てください。   
`combination_dim`は`how_many_to_save`個からいくつ選ぶ組み合わせを考えるかです。   
いくつかのオプション引数があります。nb_SO.pyをご覧ください。   

In [19]:
from nb_sisso import SO
score_list,index_list=SO(list_x,y,model_score=model_score,which_arr_to_choose_from={1:0,2:0},combination_dim=2)

2024-11-06 17:15:53,151 SO [INFO] : numba=0.60.0, numpy=2.0.2
2024-11-06 17:15:53,152 SO [INFO] : OPT=_OptLevel(3), THREADING_LAYER=default
2024-11-06 17:15:53,155 SO [INFO] : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2024-11-06 17:15:53,156 SO [INFO] : SO
2024-11-06 17:15:53,156 SO [INFO] : num_threads=8, how_many_to_save=50, 
2024-11-06 17:15:53,157 SO [INFO] : combination_dim=2, model_score=Hull_2d, 
2024-11-06 17:15:53,157 SO [INFO] : which_arr_to_choose_from={1: 0, 2: 0}
2024-11-06 17:15:53,158 SO [INFO] : loop=12497500
2024-11-06 17:15:53,158 SO [INFO] : compiling
2024-11-06 17:15:57,697 SO [INFO] : END, compile
2024-11-06 17:16:07,700 SO [INFO] :     516578/12497500  0:00:10.002708 : 0:03:51.991421
2024-11-06 17:16:17,712 SO [INFO] :    1048696/12497500  0:00:20.014513 : 0:03:38.502060
2024-11-06 17:16:27,726 SO [INFO] :    1595815/12497500  0:00:30.029148 : 0:03:25.141769
2024-11-06 17:16:37,741 SO [INFO] :    2096890/12497500  0:00:40.043552 : 0:03:18.616698
2024-11-06 

返り値はscoreと、indexになります。indexは入ってきたlist_xのindexです。

In [20]:
arr_columns=np.array([decryption(eq[i]) for i in range(eq.shape[0])])
df_ans=pd.DataFrame(columns=["score1","score2","data_x","data_y"])
df_ans["score1"]=score_list[:,0]
df_ans["score2"]=score_list[:,1]
df_ans["data_x"]=arr_columns[index_list[:,0]]
df_ans["data_y"]=arr_columns[index_list[:,1]]
df_ans

,score1,score2,data_x,data_y
0,0.504545,-1.000261e+00,((((f+f)*f)-1)**2),(f/((1/(f**3))+d))
1,0.504545,-1.000261e+00,((((f+f)*f)-1)**2),(f/(d+((1/f)**3)))
2,0.490909,-5.905357e+01,((((a*d)**2)**3)**3),(((f*(f**3))-d)*a)
3,0.490909,-5.905357e+01,((((a*d)**3)**3)**2),(((f*(f**3))-d)*a)
4,0.481818,-2.417930e+01,(f/(((f+f)*f)+1)),((d+((1/f)**3))**3)
5,0.481818,-2.417930e+01,(f/(((f+f)*f)+1)),(((1/(f**3))+d)**3)
6,0.472727,-1.355453e-01,(((1/(f+f))+f)**3),(((1/(f**3))+d)**3)
7,0.472727,-1.355453e-01,((((1/f)+f)+f)**3),((d+((1/f)**3))**3)
8,0.472727,-1.355453e-01,((((1/f)+f)+f)**3),(((1/(f**3))+d)**3)
9,0.463636,-7.095258e-01,(d/(((d+d)+d)-f)),((f/d)-((f/d)**3))
